In [1]:
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:/home/fran/IronhackMad/dataptmad1019/module-1/lab-selenium/driver'

In [ ]:
os.environ['PATH']

In [2]:
from selenium import webdriver

In [3]:
import datetime

ORIGIN = 'MADRID'
DESTINATION = 'VALENCIA'

# tickets for tomorrow
DATE = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%d/%m/%Y')

In [4]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup

browser = webdriver.Firefox()
browser.get('http://www.renfe.com/')

origin = browser.find_element_by_name('desOrigen')
origin.send_keys(ORIGIN)
origin.send_keys(Keys.ARROW_DOWN)
origin.send_keys(Keys.ENTER)

destination = browser.find_element_by_name('desDestino')
destination.send_keys(DESTINATION)
destination.send_keys(Keys.ARROW_DOWN)
destination.send_keys(Keys.ENTER)

date = browser.find_element_by_name('__fechaIdaVisual')
date.send_keys(Keys.BACK_SPACE * 10)
date.send_keys(DATE)

destination.send_keys(Keys.ENTER)

element = WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.ID, "listaTrenesTBodyIda")))
new_url = browser.current_url
print(new_url)
html = requests.get(new_url).content
soup = BeautifulSoup(html, "lxml")
table_id = browser.find_element(By.ID, 'listaTrenesTBodyIda')
rows = table_id.find_element(By.TAG_NAME, 'tr')
for i in rows:
    print(i)
#target_table = browser.find_element_by_xpath("//*[@id='listaTrenesTBodyIda']")

#print(target_table)

https://venta.renfe.com/vol/search.do?c=_Q6On


NoSuchElementException: Message: Unable to locate element: tr


In [7]:
table = soup.find_all('table', {'class' : "trayecto-sel"})

In [8]:
table

[]

In [19]:
browser.close()

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
html = requests.get(new_url).content
soup = BeautifulSoup(html, "lxml")

print(soup.find(id='listaTrenesTBodyIda'))